In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import pickle

## Import churndata

In [2]:
churn_data = pd.read_csv('Customer-Churn.csv')
churn_data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [3]:
churn_data.shape

(7043, 16)

In [4]:
#lower
churn_data.columns = [columns.lower() for columns in churn_data.columns]

In [5]:
churn_data.columns

Index(['gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

# Check the datatypes of all the columns in the data. 
# You would see that the column TotalCharges is object type. 
# Convert this column into numeric type using pd.to_numeric function.

In [6]:
churn_data.dtypes

gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [7]:
churn_data[['totalcharges']] = churn_data[['totalcharges']].apply(pd.to_numeric, errors='coerce')
churn_data[['totalcharges']]

,totalcharges
0,29.85
1,1889.50
2,108.15
3,1840.75
4,151.65
...,...
7038,1990.50
7039,7362.90
7040,346.45
7041,306.60


In [8]:
#doublecheck if numeric
churn_data['totalcharges'].dtypes

dtype('float64')

# Check for null values in the dataframe. Replace the null values.

In [9]:
churn_data.isna().sum()

gender               0
seniorcitizen        0
partner              0
dependents           0
tenure               0
phoneservice         0
onlinesecurity       0
onlinebackup         0
deviceprotection     0
techsupport          0
streamingtv          0
streamingmovies      0
contract             0
monthlycharges       0
totalcharges        11
churn                0
dtype: int64

In [10]:
def replace(churn_data, columns):
    for col in columns:
        mean_value = churn_data[columns].mean()
        churn_data[columns] = churn_data[columns].fillna(mean_value)  #fill with mean
    return churn_data

In [11]:
replace(churn_data, ['totalcharges'])

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,monthlycharges,totalcharges,churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.50,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.60,Yes


In [12]:
#doublecheck na's
churn_data['totalcharges'].isna().sum()

0

# Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
  

In [13]:
features = churn_data[['tenure', 'seniorcitizen', 'monthlycharges', 'totalcharges', 'churn']]
features

,tenure,seniorcitizen,monthlycharges,totalcharges,churn
0,1,0,29.85,29.85,No
1,34,0,56.95,1889.50,No
2,2,0,53.85,108.15,Yes
3,45,0,42.30,1840.75,No
4,2,0,70.70,151.65,Yes
...,...,...,...,...,...
7038,24,0,84.80,1990.50,No
7039,72,0,103.20,7362.90,No
7040,11,0,29.60,346.45,No
7041,4,1,74.40,306.60,Yes


In [50]:
#train-test split
X = features.drop('churn', axis =1)
y_features = features['churn']
y = y_features.apply(lambda x: 1 if x=='Yes' else 0)


In [51]:
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: churn, Length: 7043, dtype: int64

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [53]:
y_test.shape

(1761,)

In [54]:
y_test

185     1
2715    0
3825    0
1807    1
132     0
       ..
5845    1
2301    0
5121    0
677     1
6062    0
Name: churn, Length: 1761, dtype: int64

In [55]:
#X_train_num = X_train.select_dtypes(np.number)
#X_test_num = X_test.select_dtypes(np.number)

In [56]:
#scale
transformer = StandardScaler().fit(X_train)
x_train_stand = transformer.transform(X_train)

In [57]:
# pickle
with open('numerical_train.pickle', 'wb') as file:
    pickle.dump(transformer, file)

In [58]:
# load pickle  
with open('numerical_train.pickle', 'rb') as file:
    unpickled_transformer = pickle.load(file)

In [59]:
X_train_n = unpickled_transformer.transform(X_train)

In [60]:
#scale again
transformer = StandardScaler().fit(X_test)
X_test_n = transformer.transform(X_test)

In [61]:
X_train_n.shape, X_test_n.shape

((5282, 4), (1761, 4))

# Fit a logistic regression model on the training data.

In [62]:
classification_model = LogisticRegression(random_state=42, max_iter=1942)
classification_model.fit(X, y)

LogisticRegression(max_iter=1942, random_state=42)

In [63]:
#accuracy
classification_model.score(X_test, y_test)

0.7989778534923339

In [64]:
#imbalance
churn_data['churn'].value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

# downsampling

In [65]:
category_0_down = features[features['churn'] == 'No']
category_1_down = features[features['churn'] == 'Yes']

In [66]:
category_0_down = category_0_down.sample(len(category_1_down), replace = True)

In [67]:
print('Cat_0_down, Cat_1_down')
print(category_0_down.shape, category_1_down.shape)

Cat_0_down, Cat_1_down
(1869, 5) (1869, 5)


In [68]:
#check downsampling accuracy
downsample_df = pd.concat([category_0_down, category_1_down])
downsample_df

,tenure,seniorcitizen,monthlycharges,totalcharges,churn
265,61,0,88.10,5526.75,No
3585,34,1,90.15,3128.80,No
6297,64,0,113.35,7222.75,No
2698,2,0,19.85,51.60,No
4414,21,0,20.75,452.20,No
...,...,...,...,...,...
7021,12,0,59.80,727.80,Yes
7026,9,0,44.20,403.35,Yes
7032,1,1,75.75,75.75,Yes
7034,67,0,102.95,6886.25,Yes


In [69]:
X_down = downsample_df.drop('churn', axis = 1)
y_down = downsample_df['churn']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, test_size=0.25, random_state=1942)

In [71]:
classification_model_downsample = LogisticRegression(random_state=42, max_iter=1942)
classification_model_downsample.fit(X_down, y_down)

LogisticRegression(max_iter=1942, random_state=42)

In [72]:
classification_model_downsample.score(X_test, y_test) 

0.7390374331550802

In [86]:
y_test

941      No
5130    Yes
1366    Yes
4899    Yes
2148     No
       ... 
677     Yes
1469    Yes
6379    Yes
1139     No
5214    Yes
Name: churn, Length: 2070, dtype: object

# upsampling

In [73]:
category_0_up = features[features['churn'] == 'No']
category_1_up = features[features['churn'] == 'Yes']

In [74]:
category_1_up = category_1_up.sample(len(category_0_up), replace=True)

In [75]:
print('Cat_0_up, Cat_1_up')
print(category_0_up.shape, category_1_up.shape)

Cat_0_up, Cat_1_up
(5174, 5) (5174, 5)


In [76]:
#check upsampling accuracy
upsample_df = pd.concat([category_0_up, category_1_up])
upsample_df

,tenure,seniorcitizen,monthlycharges,totalcharges,churn
0,1,0,29.85,29.85,No
1,34,0,56.95,1889.50,No
3,45,0,42.30,1840.75,No
6,22,0,89.10,1949.40,No
7,10,0,29.75,301.90,No
...,...,...,...,...,...
5069,4,0,75.35,338.10,Yes
1083,2,0,80.95,171.15,Yes
5381,32,0,78.90,2447.95,Yes
3596,3,0,55.35,165.20,Yes


In [77]:
X_up = upsample_df.drop('churn', axis = 1)
y_up = upsample_df['churn']

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_up, y_up, test_size=0.2, random_state=1942)

In [79]:
classification_model_upsample = LogisticRegression(random_state=42, max_iter=1942)
classification_model_upsample.fit(X_up, y_up)

LogisticRegression(max_iter=1942, random_state=42)

In [80]:
classification_model_upsample.score(X_test, y_test) 

0.7231884057971014

In [87]:
y_test

941      No
5130    Yes
1366    Yes
4899    Yes
2148     No
       ... 
677     Yes
1469    Yes
6379    Yes
1139     No
5214    Yes
Name: churn, Length: 2070, dtype: object

# and then there is SMOTE...

In [89]:
#optional not necessary                                      #Agathe said that I need another column with the churn 0,1 values for SMOTE to work

In [90]:
#from imblearn.over_sampling import SMOTE

In [91]:
#sm = SMOTE(random_state=100,k_neighbors=3)

#X_SMOTE,y_SMOTE = sm.fit_resample(X,y)

In [92]:
#X_SMOTE.shape

In [93]:
#y_test

In [94]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score

# LR = LogisticRegression(max_iter=1000)
# LR.fit(X_SMOTE, y_SMOTE)
# pred = LR.predict(X_test)

# print("precision: ",precision_score(y_test,pred))
# print("recall: ",recall_score(y_test,pred))
# print("f1: ",f1_score(y_test,pred))